**CPLEX installation guide:** Add following line to your `.bash_profile` (for macOS) after installing IBM ILOG CPLEX Optimization Studio locally using installer from IBM. (note the path will change if you are using different python or OS)

```export PYTHONPATH="/Applications/CPLEX_Studio1210/cplex/python/3.6/x86-64_osx"```

In [1]:
import cplex

In [2]:
cplex.__path__  # make sure the path is the one we pick above

['/Applications/CPLEX_Studio1210/cplex/python/3.6/x86-64_osx/cplex']

In [3]:
import numpy as np
import pandas as pd

from util_io import (
    init, finalize, dump_conf, assigned_day_to_family_on_day, assigned_day_to_occupancy
)
from util_cost import (
    cal_total, n_people, family_id_choice_to_pref_cost, cal_total_preference, cal_total_accounting
)
from util_cost import choices as family_pref
from util_check import deep_check

In [4]:
# constants #
N_families = 5000
N_days = 100
# constants #

# params #
path_init_conf =     '../output/m08-improved.csv'
path_dump_improved = '../output/m10-improved.csv' # lowest cost

In [5]:
assigned_day, family_on_day, occupancy = init(path_conf=path_init_conf)
etotal_low = cal_total(assigned_day, occupancy)
print('Init config cost:', etotal_low)

Read initial configs...
Read config completed.
Init config cost: 71525.73988601584


In [6]:
len(assigned_day)

5000

## Ortools MIP

In [7]:
from docplex.mp.model import Model

In [8]:
model = Model('')

In [9]:
model.context.cplex_parameters.threads = 12

In [10]:
families = range(N_families)
days = range(1, N_days + 1)

In [11]:
# Possible choice for the family
# last choice is any day that is not on the family's preferred days
N_choices = family_id_choice_to_pref_cost.shape[1]
N_choices

11

In [12]:
family_id_choice_to_pref_cost

array([[   0,   50,   86, ...,  544, 1440, 2236],
       [   0,   50,   86, ...,  544, 1440, 2236],
       [   0,   50,   77, ...,  508, 1205, 1802],
       ...,
       [   0,   50,  104, ...,  616, 1910, 3104],
       [   0,   50,   95, ...,  580, 1675, 2670],
       [   0,   50,   86, ...,  544, 1440, 2236]])

## Variables

In [13]:
# Variables
# assignment matrix[family, pref_rank]
assignment_matrix = model.binary_var_matrix(families, range(N_choices), 'x')

In [14]:
len(assignment_matrix)

55000

In [15]:
possible_family_sizes = np.unique(n_people)

In [16]:
N_min_people = 125
N_max_people = 300

In [17]:
# unpreferred_day_counts[day, size]
ub = int(N_max_people / possible_family_sizes.min())
unpreferred_day_counts = model.integer_var_matrix(days, possible_family_sizes, lb=0, ub=ub, name='d')

In [18]:
len(unpreferred_day_counts)

700

In [19]:
unpreferred_day_counts[100, 8]

docplex.mp.Var(type=I,name='d_100_8',ub=150)

## Constraints

### constraint 1: each family only take one day (choice)

In [20]:
# Constraints
# constraint 1: each family only take one day (choice)
model.add_constraints_(
    [model.sum([assignment_matrix[family, c] for c in range(N_choices)]) == 1
     for family in families],
    'family_ct'
)

### constraint 2: each day can only have 125-300 people

In [21]:
# constraint 2: each day can only have 125-300 people

In [22]:
family_pref

array([[ 52,  38,  12, ...,  76,  10,  28],
       [ 26,   4,  82, ...,   6,  66,  61],
       [100,  54,  25, ...,  89,  80,  33],
       ...,
       [ 32,  66,  54, ...,  81,   3,   7],
       [ 67,  92,   4, ...,  12,  26,  70],
       [ 13,  11,  25, ...,  39,  18,  47]])

In [23]:
N_family_pref = N_choices - 1
N_family_pref

10

In [24]:
# day to dictionary of families who choice this day with value as preference rank
days_family_prefered = [{} for day in range(N_days + 1)]  # day = 0 should not be used

In [25]:
for family, pref in enumerate(family_pref):
    for pref_rank, day in enumerate(pref):
        days_family_prefered[day][family] = pref_rank

In [26]:
for day in days:
    # find those family who like this day
    family_prefered = days_family_prefered[day]
    model.add_constraint_(
        model.sum(
            [assignment_matrix[family, pref_rank] * n_people[family] 
             for family, pref_rank in family_prefered.items()]
        )
        + model.sum(
            [unpreferred_day_counts[day, s] * s for s in possible_family_sizes]
        )
        <= 300,
        'day_upper_bound[%i]' % day
    )
    model.add_constraint_(
        model.sum(
            [assignment_matrix[family, pref_rank] * n_people[family] 
             for family, pref_rank in family_prefered.items()]
        )
        + model.sum(
            [unpreferred_day_counts[day, s] * s for s in possible_family_sizes]
        )        
        >= 125,
        'day_lower_bound[%i]' % day
    )

### constraint 3: unpreferred day family count conservation for each family size

In [27]:
# constraint 3: unpreferred day family count conservation for each family size

In [28]:
family_size_to_family_ids = {
    size: np.where(n_people == size)[0] for size in possible_family_sizes
}

In [29]:
model.add_constraints_(
    [
        model.sum([assignment_matrix[family, N_choices - 1]
                   for family in family_size_to_family_ids[size]])
        == model.sum([unpreferred_day_counts[day, size] for day in days])
        for size in possible_family_sizes
    ],
    ['unpreferred_day_counts[%i]' % size for size in possible_family_sizes]
)

## Objective

In [30]:
# Objective - Preference cost only as approximation
model.minimize(
    model.sum([
        assignment_matrix[family, c] * family_id_choice_to_pref_cost[family, c]
        for family in families for c in range(N_choices)
    ])
)

xx = model.constraints()[0]

[xx.GetCoefficient(assignment_matrix[1, c]) for c in range(N_choices)]

xx.name()

## Solve

In [31]:
from docplex.mp.progress import TextProgressListener
from docplex.mp.progress import ProgressClock
model.add_progress_listener(TextProgressListener(clock=ProgressClock.Objective, absdiff=200))

In [32]:
%%time
# Solve
sol = model.solve()

  1+: Node=0 Left=1 Best Integer=269063.0000, Best Bound=43286.9286, gap=83.91%, ItCnt=399 [3.9s]
  2+: Node=0 Left=1 Best Integer=44257.0000, Best Bound=43286.9286, gap=2.19%, ItCnt=399 [4.3s]
  3+: Node=0 Left=1 Best Integer=43723.0000, Best Bound=43535.2924, gap=0.43%, ItCnt=751 [9.9s]
CPU times: user 47.7 s, sys: 1.96 s, total: 49.6 s
Wall time: 19.7 s


In [33]:
print('Solution status:', model.get_solve_status())
print('Total cost:', sol.objective_value, sol.get_objective_value())
print("Time:", '%.3f' % model.get_solve_details().time, "s")

Solution status: JobSolveStatus.OPTIMAL_SOLUTION
Total cost: 43622 43622
Time: 19.433 s


In [34]:
# for k, v in sol.iter_var_values():
#     print(k, v)

In [35]:
# sol.get_value_dict(assignment_matrix, keep_zeros=False)

In [36]:
# sol.get_value_dict(unpreferred_day_counts)

## Get solution

In [123]:
assigned_day_new = np.ones(assigned_day.shape, dtype='int32') * -1
for family, choice in sol.get_value_dict(assignment_matrix, keep_zeros=False):
    assigned_day_new[family] = family_pref[family, choice]

In [124]:
print('valid assigned_day:', (assigned_day_new >= 0).all())

valid assigned_day: True


In [128]:
unpreferred_day_counts_sol_dict = sol.get_value_dict(unpreferred_day_counts)

In [129]:
# if N_choices >= N_choices_ori:
unpreferred_day_counts_sol = {
    size: [0]+[int(unpreferred_day_counts_sol_dict[day, size]) for day in days]
    for size in possible_family_sizes
}

print('Unpreferred families slots:')
{size: sum(counts) for size, counts in unpreferred_day_counts_sol.items()}

Unpreferred families slots:


{2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0}

In [84]:
family_on_day_new = assigned_day_to_family_on_day(assigned_day_new)

occupancy_new = assigned_day_to_occupancy(assigned_day_new)

deep_check(assigned_day_new, family_on_day_new, occupancy_new)

deep check: everything looks fine.


True

In [87]:
print('Total score:    ', cal_total(assigned_day_new, occupancy_new))
print('Preference cost:', cal_total_preference(assigned_day_new))
print('Accounting cost:', cal_total_accounting(occupancy_new))

Total score:     16371275794.971413
Preference cost: 43625
Accounting cost: 16371232169.971413


In [85]:
occupancy_new

array([125, 300, 197, 300, 300, 300, 138, 172, 173, 174, 299, 300, 300,
       164, 213, 197, 218, 300, 299, 300, 158, 143, 164, 190, 300, 300,
       299, 237, 181, 227, 132, 300, 300, 300, 125, 140, 140, 125, 293,
       298, 298, 139, 128, 148, 132, 300, 298, 299, 170, 154, 127, 159,
       300, 296, 300, 125, 129, 125, 173, 299, 300, 286, 125, 125, 125,
       125, 300, 297, 283, 126, 125, 125, 125, 295, 236, 300, 125, 125,
       125, 125, 255, 300, 280, 125, 125, 125, 126, 299, 300, 288, 125,
       128, 125, 125, 294, 293, 297, 125, 125, 125, 125, 125], dtype=int32)

## Output solution

In [86]:
dump_conf(assigned_day_new, path_dump_improved)

## Debug

In [29]:
# [
#     [assignment_matrix[family, c].solution_value() for c in range(N_choices)]
#     for family in range(10)
# ]        

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]

In [30]:
# [
#     [unpreferred_day_counts[day, size].solution_value() for size in possible_family_sizes]
#     for day in range(1, 10)
# ]        

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]